# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **FocalLoss** for class-imbalanced learning

**Target Metrics:**
- Specificity: 98.72%
- G-Means Improvement: 18.11%
- P95 Latency: <100ms

## 1️⃣ Setup Environment

In [1]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

Cloning into 'fraudguard'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 162 (delta 62), reused 135 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (162/162), 117.11 KiB | 1.67 MiB/s, done.
Resolving deltas: 100% (62/62), done.
/content/fraudguard


In [3]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

import torch

# 1. Get exact versions
pt_version = torch.__version__.split('+')[0]  # e.g., 2.5.1
cuda_version = "cu" + torch.version.cuda.replace('.', '')  # e.g., cu124
wheel_url = f"https://data.pyg.org/whl/torch-{pt_version}+{cuda_version}.html"

print(f"PyTorch: {pt_version}, CUDA: {cuda_version}")
print(f"Downloading from: {wheel_url}")

# 2. Install with visible output (force reinstall to fix broken partial installs)
!pip install --force-reinstall torch-scatter torch-sparse -f $wheel_url

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.7 MB/s eta 0:00:00
⚠️ faiss-gpu not available, using faiss-cpu
   (Graph building on CPU, but GNN training still runs on GPU!)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 112.4 MB/s eta 0:00:00
PyTorch: 2.9.0, CUDA: cu126
Looking in links: https://data.pyg.org/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 126.1 MB/s eta 0:00:00


Obtaining file:///content/fraudguard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 7.3 MB/s eta 0:00:00
  Building editable for fraudguard (pyproject.toml) ... done
  Created wheel for fraudguard: filename=fraudguard-0.1.0-py3-none-any.whl size=2801 sha256=5638c0b714acd4b297203f70f102910966a1da4432de3a3f589a22ae37244e19
  Stored in directory: /tmp/pip-ephem-wheel-cache-borqex6g/wheels/c6/29/62/fb6d8d095576e7e3efddf4fdcb7dfc799af71ace273f1ee84c
Successfully built fraudguard
  Attempting uninstall: structlog
    Found existing installation: structlog 25.5.0
    Uninstalling structlog-25.5.0:
      Successfully unins


✓ Environment setup complete


In [1]:
import torch
try:
    import torch_scatter
    import torch_sparse
    import fraudguard
    print("✅ Success! Libraries are installed and loaded.")
except ImportError as e:
    print(f"❌ Still missing libraries: {e}")
    # Only if you see this error should you go back and install again.

✅ Success! Libraries are installed and loaded.


## 2️⃣ Configuration

In [7]:
import os

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 1.0      # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 30
BATCH_SIZE = 4096      # Reduce to 2048 or 1024 if OOM
NUM_NEIGHBORS = [25, 10]  # 2-hop neighborhood sampling

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

Data: /content/drive/MyDrive/ieee-fraud-detection
Models: /content/drive/MyDrive/fraudguard-models
Logs: /content/drive/MyDrive/fraudguard-logs

Batch size: 4096
Sample fraction: 100%


## 3️⃣ Verify GPU and FAISS

In [8]:
import torch
import faiss

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
VRAM: 15.8 GB

✓ GNN training will run on GPU

FAISS GPUs: 0
   (Using CPU FAISS for graph building - this is OK)


## 4️⃣ Load and Preprocess Data

In [9]:
import sys
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from src.data.loader import FraudDataLoader
from src.data.preprocessor import FeaturePreprocessor
from src.data.graph_builder import GraphBuilder
from src.utils.config import load_data_config
from src.utils.device_utils import set_seed, get_device

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

Using device: cuda

Data loaded:
  Train: 354,324
  Val: 118,108
  Test: 118,108
  Fraud rate: 3.50%


## 5️⃣ Run Full AD-RL-GNN Pipeline

We use the `FraudTrainer` class to orchestrate the full pipeline, including:
1. **AdaptiveMCD**: Intelligent majority downsampling
2. **RL Agent**: Dynamic subgraph selection (Random Walk, K-Hop, K-Ego)
3. **Graph Enhancement**: Adding semantic edges
4. **GNN Training**: CrossEntropyLoss (15x weight)

# Vanilla Baseline GNN

In [10]:
import torch
import gc
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from src.training.trainer import FraudTrainer
from src.utils.config import load_model_config, load_data_config
from sklearn.metrics import confusion_matrix
import numpy as np
import time

model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)


# Aligned Hyperparameters (Strict Ceteris Paribus)
model_cfg.training["max_epochs"] = 30
model_cfg.training["learning_rate"] = 0.003
model_cfg.adaptive_mcd["alpha"] = 0.0        # No MCD
model_cfg.graph.similarity_threshold = 0.75

print("Initializing Vanilla Baseline (No MCD, No RL)...")
trainer = FraudTrainer(model_config=model_cfg, data_config=data_cfg, device=device)

# Load 100% Data
loader = FraudDataLoader(config=data_cfg)
df_full = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df_full)

# Preprocess
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

# Reset VRAM Monitor
torch.cuda.reset_peak_memory_stats()

# Apply Weight 25.0
weights = torch.tensor([1.0, 25.0]).to(device)
trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
model = trainer.model
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)

# Loaders (Batch 2048)
optimized_data = Data(x=trainer.X_full, edge_index=trainer.edge_index, y=trainer.all_labels)
optimized_data.train_mask = trainer.train_mask; optimized_data.val_mask = trainer.val_mask
train_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=2048, input_nodes=optimized_data.train_mask, shuffle=True)
val_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=2048, input_nodes=optimized_data.val_mask, shuffle=False)

print(f"\n Starting Baseline Training (30 Epochs)...")
best_gmeans_baseline = 0

for epoch in range(30):
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = trainer.criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

        # Gradient clipping
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    # Eval every 1 epoch
    if epoch % 1 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)
        print(f"Baseline Epoch {epoch+1:>2} | Spec: {tnr*100:.2f}% | Recall: {tpr*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans_baseline:
            best_gmeans_baseline = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_baseline.pt")

# Capture Baseline Metrics
baseline_vram = torch.cuda.max_memory_allocated() / 1e9

model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_baseline.pt"))
model.eval()
latencies_baseline = []
with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        _ = model(batch.x, batch.edge_index)
        latencies_baseline.append((time.perf_counter() - start) * 1000)

baseline_p95 = np.percentile(latencies_baseline, 95)

print(f"\n Baseline VRAM: {baseline_vram:.2f} GB")
print(f" Baseline P95 Latency: {baseline_p95:.2f} ms")
print(f" Baseline Best G-Means: {best_gmeans_baseline*100:.2f}%")

# Clean up
del model, trainer, optimized_data, train_loader, val_loader
torch.cuda.empty_cache()
gc.collect()

🚀 Initializing VANILLA BASELINE (No MCD, No RL)...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(



📉 Starting Baseline Training (30 Epochs, Aligned)...
Baseline Epoch  1 | Spec: 85.07% | Recall: 16.31% | G-Means: 37.25%
Baseline Epoch  2 | Spec: 94.45% | Recall: 8.05% | G-Means: 27.57%
Baseline Epoch  3 | Spec: 95.49% | Recall: 6.64% | G-Means: 25.17%
Baseline Epoch  4 | Spec: 92.09% | Recall: 11.52% | G-Means: 32.57%
Baseline Epoch  5 | Spec: 89.78% | Recall: 14.99% | G-Means: 36.68%
Baseline Epoch  6 | Spec: 76.97% | Recall: 27.50% | G-Means: 46.01%
Baseline Epoch  7 | Spec: 91.98% | Recall: 12.06% | G-Means: 33.30%
Baseline Epoch  8 | Spec: 80.81% | Recall: 25.57% | G-Means: 45.46%
Baseline Epoch  9 | Spec: 85.23% | Recall: 20.78% | G-Means: 42.08%
Baseline Epoch 10 | Spec: 85.96% | Recall: 19.95% | G-Means: 41.41%
Baseline Epoch 11 | Spec: 94.62% | Recall: 10.30% | G-Means: 31.22%
Baseline Epoch 12 | Spec: 88.61% | Recall: 17.52% | G-Means: 39.41%
Baseline Epoch 13 | Spec: 93.70% | Recall: 11.26% | G-Means: 32.48%
Baseline Epoch 14 | Spec: 93.58% | Recall: 11.58% | G-Means: 32.

70

# Improved AD-RL-GNN

In [11]:
model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Aligned Hyperparameters (Strict Ceteris Paribus)
model_cfg.training["max_epochs"] = 30
model_cfg.training["learning_rate"] = 0.003
model_cfg.rl_agent["reward_scaling"] = 2.0
model_cfg.adaptive_mcd["alpha"] = 0.80       # Aggressive cleaning
model_cfg.graph.similarity_threshold = 0.75

print(f"\n Initializing AD-RL (MCD=ON, RL=ON)...")
trainer = FraudTrainer(model_config=model_cfg, data_config=data_cfg, device=device)

# Reset Stats
torch.cuda.reset_peak_memory_stats()

# Re-process
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

print("\n Training AdaptiveMCD (Alpha 0.80)...")
trainer._train_mcd()

print("\n Training RL Agent...")
trainer._train_rl_and_enhance()

# VRAM Flush
print("\n🧹 Flushing VRAM before GNN Training...")
torch.cuda.empty_cache()
gc.collect()

# Weight 25.0
weights = torch.tensor([1.0, 25.0]).to(device)
trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
model = trainer.model
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)

# Loaders (Batch 2048)
optimized_data = Data(x=trainer.X_full, edge_index=trainer.edge_index, y=trainer.all_labels)
optimized_data.train_mask = trainer.train_mask; optimized_data.val_mask = trainer.val_mask
train_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=2048, input_nodes=optimized_data.train_mask, shuffle=True)
val_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=2048, input_nodes=optimized_data.val_mask, shuffle=False)

print(f"\n Starting AD-RL Training (30 Epochs)...")
best_gmeans_gold = 0

for epoch in range(30):
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = trainer.criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

        # Gradient clipping
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    # Eval every 1 epoch
    if epoch % 1 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)
        print(f"Epoch {epoch+1:>3} | Spec: {tnr*100:.2f}% | Recall: {tpr*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans_gold:
            best_gmeans_gold = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_final_aggressive.pt")

# Capture AD-RL Metrics
gold_vram = torch.cuda.max_memory_allocated() / 1e9

model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_final_aggressive.pt"))
model.eval()
latencies_gold = []
with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        _ = model(batch.x, batch.edge_index)
        latencies_gold.append((time.perf_counter() - start) * 1000)

gold_p95 = np.percentile(latencies_gold, 95)

print(f"\n✅ Gold VRAM: {gold_vram:.2f} GB")
print(f"✅ Gold P95 Latency: {gold_p95:.2f} ms")
print(f"🏁 Final Best G-Means: {best_gmeans_gold*100:.2f}%")

# Clean up
del model, trainer, optimized_data, train_loader, val_loader
torch.cuda.empty_cache()
gc.collect()


🚀 Initializing AGGRESSIVE GOLD RUN (RL=ON)...

🧠 Training AdaptiveMCD (Alpha 0.80)...

🤖 Training RL Agent...

🧹 Flushing VRAM before GNN Training...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(



🚀 Starting Gold Standard Training (30 Epochs)...
Epoch   1 | Spec: 28.01% | Recall: 74.58% | G-Means: 45.71%
Epoch   2 | Spec: 13.55% | Recall: 88.90% | G-Means: 34.70%
Epoch   3 | Spec: 29.52% | Recall: 75.86% | G-Means: 47.32%
Epoch   4 | Spec: 28.00% | Recall: 78.83% | G-Means: 46.98%
Epoch   5 | Spec: 24.96% | Recall: 81.11% | G-Means: 45.00%
Epoch   6 | Spec: 29.47% | Recall: 78.12% | G-Means: 47.98%
Epoch   7 | Spec: 37.43% | Recall: 70.14% | G-Means: 51.23%
Epoch   8 | Spec: 47.85% | Recall: 61.68% | G-Means: 54.33%
Epoch   9 | Spec: 44.87% | Recall: 63.09% | G-Means: 53.20%
Epoch  10 | Spec: 43.56% | Recall: 67.47% | G-Means: 54.21%
Epoch  11 | Spec: 52.83% | Recall: 56.97% | G-Means: 54.86%
Epoch  12 | Spec: 54.29% | Recall: 54.63% | G-Means: 54.46%
Epoch  13 | Spec: 47.20% | Recall: 63.78% | G-Means: 54.87%
Epoch  14 | Spec: 45.19% | Recall: 65.86% | G-Means: 54.55%
Epoch  15 | Spec: 64.64% | Recall: 47.73% | G-Means: 55.55%
Epoch  16 | Spec: 55.04% | Recall: 54.80% | G-Mean

## 6️⃣ Evaluation & Claims Verification

In [1]:
# ==============================================================================
# 3. FINAL A/B TEST REPORT
# ==============================================================================
print("\n" + "="*60)
print("🎯 FINAL ARCHITECTURAL COMPARISON (Scientifically Aligned)")
print("="*60)
print(f"| Metric      | Baseline | Optimized | Improvement |")
print(f"|-------------|----------|-----------|-------------|")
print(f"| G-Means     | {best_gmeans_baseline*100:.1f}%    | {best_gmeans_gold*100:.1f}%     | +{((best_gmeans_gold-best_gmeans_baseline)/best_gmeans_baseline)*100:.1f}%        |")
print(f"| P95 Latency | {baseline_p95:.1f} ms  | {gold_p95:.1f} ms   | {((baseline_p95-gold_p95)/baseline_p95)*100:.1f}%         |")
print(f"| Peak VRAM   | {baseline_vram:.1f} GB   | {gold_vram:.1f} GB    | {((baseline_vram-gold_vram)/baseline_vram)*100:.1f}%         |")
print("="*60)


🎯 FINAL ARCHITECTURAL COMPARISON (Scientifically Aligned)
| Metric      | Baseline | Optimized | Improvement |
|-------------|----------|-----------|-------------|


NameError: name 'best_gmeans_baseline' is not defined

## 7️⃣ Save Model

In [2]:
trainer.save(f"{MODELS_DIR}/fraudguard_full_pipeline.pt")
print(f"Model saved to {MODELS_DIR}/fraudguard_full_pipeline.pt")

NameError: name 'trainer' is not defined

In [ ]:
from google.colab import runtime
runtime.unassign()